# Material Protocol Buffer Usage

This notebook will show you how to use the protocol buffer for materials for BART using the python interface. 

Dependencies:
- python3

_Optional:_ recompile the protocol buffer.

In [2]:
!protoc --python_out=./proto -I=../proto ../proto/material_protobuf.proto 

Import the generated protocol buffer package.

In [1]:
import proto.material_protobuf_pb2 as mat_pb

We will be making a new material library, each `message` in the `.proto` file is an object, so we will instantiate a new one. `Library()` is in the top level of the package.
```
message Library {
```

In [2]:
library = mat_pb.Library()

## Top level fields

We can now set the single fields in our `Library()`:
```
string library_name = 1;
uint32 number_of_groups = 2;
  ```

In [3]:
library.library_name = "Two-group test library"
library.number_of_groups = 2

The energy_groups field has the `repeated` codeword, so it requires a list. It starts as an empty list.
```
repeated float energy_groups = 3 [packed = true];
  ```

In [4]:
library.energy_groups

[]

There are various ways to set it. You can use `.append()` to add one value, or `.extend()` to add a list, or other accessors, the `[:]` accessor allows you to set the whole list:

In [5]:
library.energy_groups[:] = ([0.025, 1e6])
library.energy_groups

[0.02500000037252903, 1000000.0]

We can review the library object:

In [6]:
library

library_name: "Two-group test library"
number_of_groups: 2
energy_groups: 0.02500000037252903
energy_groups: 1000000.0

## Making a new material

The `repeated` field for materials holds Material objects, so we need to instantiate them using the `.add()` command:
```
repeated Material materials = 4;
```

In [7]:
u235 = library.materials.add()

This `u233` is now an instantiation of the Material object (it's a `message` so it's an object)
```
message Material {
```

We can set the first four fields easily:
```
string full_name = 1;
string abbreviation = 2;
uint32 A = 3;
uint32 Z = 4;
    ```

In [8]:
u235.full_name = "Uranium 235"
u235.abbreviation = "U235"
u235.A = 235
u235.Z = 91

We can check that we've set the right values and always change them if need be.

In [9]:
u235

full_name: "Uranium 235"
abbreviation: "U235"
A: 235
Z: 91

In [10]:
u235.Z =92
u235

full_name: "Uranium 235"
abbreviation: "U235"
A: 235
Z: 92

## Adding Properties

There are three types of properties that can be added.

- Scalar quantities, such as density or other material properties.
- Vector quantities, such as $\chi$ or $\Sigma_t$ that have different values for each energy group.
- Matrix quantities, such as $\Sigma_s$ that are usually $G \times G$ where $G$ is the number of energy groups.

Each property is a `message` in one of three repeated fields:
```
repeated Scalar_Property scalar_properties = 5;
repeated Vector_Property vector_properties = 6;
repeated Matrix_Property matrix_properties = 7;
    ```
So they need to be added using `.add()` and then their properties added. There are `enum`s that hold IDs to identify possible properties.

For example, to add a density scalar quantity:
```
message Scalar_Property {
  Scalar_ID id = 1;
  float value = 2;
}
```
We need to look up the density id:
```
  enum Scalar_ID {
    UNKNOWN_SCALAR = 0;
    DENSITY = 1;
    ...
```

In [11]:
density = u235.scalar_property.add()

In [12]:
density.id = library.DENSITY
density.value = 9.8

In [13]:
u235

full_name: "Uranium 235"
abbreviation: "U235"
A: 235
Z: 92
scalar_property {
  id: DENSITY
  value: 9.800000190734863
}

The process is similar for vectors:
```
enum Vector_ID {
  UNKNOWN_VECTOR = 0;
  SIGMA_T = 1;
  CHI = 2;
  ...
}
...

message Vector_Property {
  Vector_ID id = 1;
  repeated double values = 2 [packed=true];
}
```


In [14]:
sigma_t = u235.vector_property.add()

In [16]:
sigma_t.id = library.SIGMA_T
sigma_t.value[:] = [1.0, 2.0]

In [17]:
u235

full_name: "Uranium 235"
abbreviation: "U235"
A: 235
Z: 92
scalar_property {
  id: DENSITY
  value: 9.800000190734863
}
vector_property {
  id: SIGMA_T
  value: 1.0
  value: 2.0
}

In [18]:
sigma_s = u235.matrix_property.add()

In [23]:
sigma_s.id = library.SIGMA_S
sigma_s.rows = 2
sigma_s.cols = 2
sigma_s.value[:] = [1.0, 0.0, 0.0, 1.0]

In [24]:
u235

full_name: "Uranium 235"
abbreviation: "U235"
A: 235
Z: 92
scalar_property {
  id: DENSITY
  value: 9.800000190734863
}
vector_property {
  id: SIGMA_T
  value: 1.0
  value: 2.0
}
matrix_property {
  id: SIGMA_S
  rows: 2
  cols: 2
  value: 1.0
  value: 0.0
  value: 0.0
  value: 1.0
}

In [26]:
chi = u235.vector_property.add()

In [29]:
chi.id = library.CHI
chi.value[:] =  [0.0, 0.5]

In [33]:
del u235.vector_property[1]

In [34]:
u235

full_name: "Uranium 235"
abbreviation: "U235"
A: 235
Z: 92
scalar_property {
  id: DENSITY
  value: 9.800000190734863
}
vector_property {
  id: SIGMA_T
  value: 1.0
  value: 2.0
}
vector_property {
  id: CHI
  value: 0.0
  value: 0.5
}
matrix_property {
  id: SIGMA_S
  rows: 2
  cols: 2
  value: 1.0
  value: 0.0
  value: 0.0
  value: 1.0
}

In [36]:
f = open('material_library', 'wb')
f.write(library.SerializeToString())
f.close()